In [11]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'MyHomework/HW4/'
FOLDERNAME = 'MSML641/Homework-4/' # ADJUST THIS
assert FOLDERNAME is not None, "[!] Enter the foldername."

%cd /content/drive/My\ Drive/$FOLDERNAME

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/MSML641/Homework-4


In [12]:
!pip install llama_index
!pip install llama-index-llms-huggingface llama-index-embeddings-huggingface
# !pip install -U bitsandbytes
# !pip install -U bitsandbytes accelerate
!pip install jedi
!pip install -U bitsandbytes accelerate transformers
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
from llama_index.core import Settings, Document, VectorStoreIndex
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from huggingface_hub import login
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
import torch
from transformers import BitsAndBytesConfig
import bitsandbytes as bnb
from google.colab import userdata
!pip install llama_index-experimental
from llama_index.experimental.query_engine import PandasQueryEngine
import bitsandbytes
print(f"Bitsandbytes version: {bitsandbytes.__version__}")

import torch
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("WARNING: You are on a CPU runtime. Bitsandbytes will NOT work.")

HuggingFaceToken = userdata.get('HugginfaceToken')

login(token=HuggingFaceToken)

df = pd.read_csv('IMDB_dataset.csv')
df

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.5/88.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.3/191.3 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.3/213.3 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: httpx-sse
    Found existing installation: httpx-sse 0.4.3
    Uninstalling httpx-sse-0.4.3:
      Successfully uninstalled httpx-sse-0.4.3


Bitsandbytes version: 0.48.2
CUDA Available: True
GPU Name: Tesla T4


,Title,Year,Genres,Certificate,Runtime,Rating,Metascore,Votes,Gross(Million),Director,Stars,Summary
0,The Shawshank Redemption,1994,Drama,R,142.0,9.3,82.0,2764512,28.34,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...","Over the course of several years, two convicts..."
1,The Dark Knight,2008,"Action, Crime, Drama",PG-13,152.0,9.0,84.0,2737769,534.86,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker wreaks havo...
2,Inception,2010,"Action, Adventure, Sci-Fi",PG-13,148.0,8.8,74.0,2429452,292.58,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",A thief who steals corporate secrets through t...
3,Fight Club,1999,Drama,R,139.0,8.8,67.0,2201448,37.03,David Fincher,"Brad Pitt, Edward Norton, Meat Loaf, Zach Grenier",An insomniac office worker and a devil-may-car...
4,Forrest Gump,1994,"Drama, Romance",PG-13,142.0,8.8,82.0,2150299,330.25,Robert Zemeckis,"Tom Hanks, Robin Wright, Gary Sinise, Sally Field",The history of the United States from the 1950...
...,...,...,...,...,...,...,...,...,...,...,...,...
9994,The Hairdresser's Husband,1990,"Comedy, Drama, Romance",R,82.0,7.2,NaN,10122,1.16,Patrice Leconte,"Jean Rochefort, Anna Galiena, Roland Bertin, M...",Antoine has always been fascinated by a hairdr...
9995,Rock-A-Doodle,1991,"Animation, Adventure, Comedy",G,77.0,6.0,NaN,10119,11.66,Don Bluth,"Gary Goldman, Dan Kuenster, Glen Campbell, Chr...",In order to defeat the wicked Grand Duke of Ow...
9996,Son of Sardaar,2012,"Action, Comedy, Romance",Not Rated,141.0,4.2,NaN,10118,0.71,Ashwani Dhir,"Anil Devgan, Ajay Devgn, Sanjay Dutt, Sonakshi...",A guest tries to stay inside the house of a ma...
9997,Heropanti,2014,"Action, Romance",Not Rated,146.0,5.2,NaN,10117,0.06,Sabir Khan,"Tiger Shroff, Kriti Sanon, Parthaa Akerkar, Su...",Two young people find love despite the violent...


In [13]:
print(df.isnull().sum())

Title                0
Year                 0
Genres               0
Certificate        374
Runtime              2
Rating               0
Metascore         1949
Votes                0
Gross(Million)    2741
Director             0
Stars                3
Summary              0
dtype: int64


In [14]:
## Data Cleaning
df['Certificate'] = df['Certificate'].fillna('Unknown')
# df['Runtime'] = df['Runtime'].fillna(-1)
df['Runtime'] = df['Runtime'].dropna()
# df['Metascore'] = df['Metascore'].fillna(-1)
df['Metascore'] = df['Metascore'].dropna()
df['gross_was_missing'] = df['Gross(Million)'].isnull()
df['Gross(Million)'] = df['Gross(Million)'].fillna(0)
df['Stars'] = df['Stars'].fillna('Unknown')
print(df["Year"].dtype)

int64


In [15]:
print(df.isnull().sum())

Title                   0
Year                    0
Genres                  0
Certificate             0
Runtime                 2
Rating                  0
Metascore            1949
Votes                   0
Gross(Million)          0
Director                0
Stars                   0
Summary                 0
gross_was_missing       0
dtype: int64


SETUP THE SENTENCE TRANSFORMERS

In [16]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

Settings.llm = HuggingFaceLLM(
    model_name="Qwen/Qwen2.5-7B-Instruct",
    tokenizer_name="Qwen/Qwen2.5-7B-Instruct",
    context_window=4096,
    max_new_tokens=256,
    model_kwargs={"quantization_config": quantization_config},
    generate_kwargs={"temperature": 0.1},
    device_map="auto",
)


Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

# 3. Create Documents for Vector Index
# We combine title and overview for semantic search
documents = []
for index, row in df.iterrows():
    text_content = f"Title: {row['Title']}\nPlot: {row['Summary']}\nGenre: {row['Genres']}"
    # We store metadata for filtering if needed later
    documents.append(Document(text=text_content, metadata={"title": row['Title']}))

# 4. Build Index (This takes time!)
vector_index = VectorStoreIndex.from_documents(documents)
semantic_engine = vector_index.as_query_engine(similarity_top_k=3)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [17]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(df['Summary'])

print("--- Results (Dense Embedding) ---")
print(f"Number of sentences: {embeddings.shape[0]}")
print(f"Embedding dimension (vector size): {embeddings.shape[1]}")
print("\nFirst 5 values of the first sentence's embedding:")
print(embeddings[0][:5])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

--- Results (Dense Embedding) ---
Number of sentences: 9999
Embedding dimension (vector size): 384

First 5 values of the first sentence's embedding:
[-4.6312883e-02 -4.1786221e-05 -4.9234238e-02  3.1948183e-02
 -2.8361857e-02]


**Semantic Query Implementation**

In [18]:
# Test Semantic Query
print("What are some movies about time travel paradoxes?")
response = semantic_engine.query("What are some movies about time travel paradoxes?")
print(str(response))

# Test Semantic Query
print("What are some movies with Bratt Pitt in them?")
response = semantic_engine.query("What are some movies with Bratt Pitt in them?")
print(str(response))

# Test Semantic Query
print("Which movies are the most action packed?")
response = semantic_engine.query("Which movies are the most action packed?")
print(str(response))

# Test Semantic Query
print("Which movies are best for kids?")
response = semantic_engine.query("Which movies are best for kids?")
print(str(response))

# Test Semantic Query
print("Which movies are based on true story?")
response = semantic_engine.query("Which movies are based on true story?")
print(str(response))

What are some movies about time travel paradoxes?
1. "The Cloverfield Paradox" - While orbiting a planet on the brink of war, scientists test a device to solve an energy crisis and end up face-to-face with a dark alternate reality.
2. "Frequently Asked Questions About Time Travel" - Three social outcasts attempt to navigate a time-travel conundrum while drinking at their local pub.
3. "The Time Machine" - A 19th-century inventor travels 800,000 years into the future, where they find humankind divided into two warring races, hoping to alter the events of the past. 
These movies explore different aspects of time travel paradoxes, from scientific experiments gone wrong to social outcasts trying to solve time-related problems and a futuristic adventure. All three fall under the sci-fi genre.
What are some movies with Bratt Pitt in them?
 The given context information does not include any movies featuring Brad Pitt. Therefore, I cannot provide a direct answer based on the provided details. 

Factual Query Implementation

In [23]:
# Create the Analyst
factual_engine = PandasQueryEngine(
    df=df,
    verbose=True,
    instruction_str="Generate only the Python pandas code to answer the query. Do NOT include any comments, explanations, or markdown formatting (e.g., backticks). Use the provided dataframe columns. For example: `df['Column'].mean()`"
)

# Test Factual Query
response = factual_engine.query("What is the average score of movies released after 2015?")
print(str(response))

response = factual_engine.query("What are some scary movies about ghosts?")
print(str(response))

response = factual_engine.query("What are some movies starring Brad Pitt")
print(str(response))

response = factual_engine.query("What the top 10 highest rated movies?")
print(str(response))

response = factual_engine.query("Which movies are best for kids?")
print(str(response))

> Pandas Instructions:
```
 df[df['Year'] > 2015]['Rating'].mean()
```
> Pandas Output: 6.4072800357302375
6.4072800357302375
> Pandas Instructions:
```
 df[df['Genres'].str.contains('Horror|Ghost', case=False)]['Title']
```
> Pandas Output: 56                   The Shining
74                         Alien
                  ...           
9990    Don't Torture a Duckling
9992                  Wilderness
Name: Title, Length: 1283, dtype: object
56                   The Shining
74                         Alien
                  ...           
9990    Don't Torture a Duckling
9992                  Wilderness
Name: Title, Length: 1283, dtype: object
> Pandas Instructions:
```
 df[df['Stars'].str.contains('Brad Pitt')]
```
> Pandas Output:            Title  Year                      Genres Certificate  Runtime  \
3     Fight Club  1999                       Drama           R    139.0   
13         Se7en  1995       Crime, Drama, Mystery           R    127.0   
...          ...   ...        

**Query Classification and Integration**

In [25]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

# Define tools
semantic_tool = QueryEngineTool(
    query_engine=semantic_engine,
    metadata=ToolMetadata(
        name="semantic_search",
        description="Useful for answering questions about movie plots, themes, characters, or finding movie recommendations."
    )
)

factual_tool = QueryEngineTool(
    query_engine=factual_engine,
    metadata=ToolMetadata(
        name="data_analysis",
        description="Useful for statistical questions, calculating averages, finding highest/lowest values, counting movies, or specific dates/years."
    )
)

# Initialize Router
# LLMSingleSelector asks the LLM to choose the best tool based on the description
query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[semantic_tool, factual_tool],
    verbose=True
)

# Unified Interface
def ask_movie_bot(question):
    print(f"Query: {question}")
    response = query_engine.query(question)
    print(f"Answer: {str(response)}\n" + "-"*30)

# Final Testing
ask_movie_bot("What are some scary movies about ghosts?") # Should route to semantic

question = ""
while question != "quit":
  question = input("Enter your question: ")
  ask_movie_bot(question)

Query: What are some scary movies about ghosts?
Selecting query engine 0: The question is asking for a list of movies with a specific theme (ghosts), which falls under the category of movie recommendations and information about movie plots, characters, or themes..
Answer: 1. Ghost Stories - A drama and horror movie where a skeptical professor investigates three unexplained cases of apparitions.
2. Paranormal Activity: The Ghost Dimension - A horror and mystery movie about a family protecting their daughter from an evil entity that can be seen by a special camera. 
3. Scary Movie 4 - Although primarily a comedy, this movie features a subplot involving a haunted house and a little boy ghost, making it a potential choice for those seeking a lighter take on the ghost genre. However, it's important to note that Scary Movie 4 is primarily a comedy and may not be as intense or serious as the other two options. It also includes an unrelated plot about alien "Tr-iPods" invading the world. There

KeyboardInterrupt: Interrupted by user